## Lecture : Flash messages

When we need to send message to our users like popping up message. And, falsh messages actually use the users session, and, sessions in Flask are cryptographically signed, which is awesome, but, that means that we have to provide, a secret key for Flask to use to sign our messages.


<pre>
  1 <!DOCTYPE html>
  2 <html>
  3     <head>
  4         <title>Character Generator</title>
  5         <link rel="stylesheet" href="/static/css/normalize.min.css">
  6         <link rel="stylesheet" href="/static/css/main.css">
  7         <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
  8         <meta name="viewport" content="width=device-width, initial-scale=1">
  9     </head>
 10     <body>
 11         <div class="wrap no-bottom messages">
 12             {% with messages = get_flashed_messages() %}
 13
 14             {% endwith %}
 15         </div>
 16         {% block content %}{% endblock %}
 17     </body>
 18 </html>
 </pre>

inside the with paragraph, have variable called messages which is from get_flashed_messages() that gets all of our flash messages

flash(): This function stores a message in the session that will self-destruct after the response is returned.

get_flashed_messages(): This function gets all of the flash messages stored in the session.

app.secret_key: This configuration attribute stores the secret key that all messages are cryptographically signed with.

{% with %}: The Flask template version of Python's with block. Let's you temporarily define a variable. Must be closed with {% endwith %}.

** Heroku : deploying flask  on web**

https://devcenter.heroku.com/articles/getting-started-with-python#introduction

# Build a Social Network with Flask
- use flask, peewee + others
- covers user registration, sign ups, and user authentication, or log ins
- cryptographically hashin passwords with a decrypt library
- function to follow or unfollow people

- Third party log ins using tweeter and facebook. 
- The hashing library that we use produce hashes of around like 60 characters. But we want to have some wiggle room just in case something changes.

** put in all caps ** doesn't make it a constant, it tells people that you're wanting this to be a constant.

in /models.py

<pre>
1 import datetime
  2
  3 from peewee import *
  4
  5 DATABASE = SqliteDatabase('social.db')
  6
  7
  8 class User(Model):
  9     username = CharField(unique=True)
 10     email = CharField(unique=True)
 11     password = CharField(max_length=100)
 12     joined_at = DataTimeField(default=datetime.datetime.now)
 13     is_admin = BooleanField(default=False) # for extention, can add more on admin user
 14
 15     class Meta:
 16         database = DATABASE
 17         order_by = ('-joined_at',) # - sign for descending, newer come first
 
</pre>

packages : flask-login (we just use user mixin which is a class that gives some small in scope piece of functionality that isn't standalone.In this case, it gives us a few properties to tell if user is logged in or not, and a method for retrieving the user's ID. This won't change the actual database table though.

 from flask.ext.login import UserMixin
 
we use pip install flask-login but the library import like this because the flask ecosystem has this kind of neat pattern where the packages get installed to this ext module(stands for external area).

9 class User(UserMixin, Model):

we can use 2 parent class like this. latter one is the ultimate parent class.

https://flask-login.readthedocs.io/en/latest/#your-user-class
** The class that you use to represent users needs to implement these properties and methods: **
- is_authenticated
- is_active
- is_anonumous
- get_id()



## Lecture : Cryptographic Hashing with Flask-Bcrypt
- bcrypt lib : uses the Blowfish cipher salt to prevent rainbow table attacks and is resistant to brute force attacks.
- Cryptographic Hashing : encryption is what you do when you create a secret message to send to your friend. You swap a for m, b for n, c for o etxetera, and do the opposite to read it. Encryption can always be reversed if you know the process, or have the correct key. Hashing, though, is a process that can't be undone. Without going too far into in, hashing involves a hash function that will always turn the same input into the same output, while in the same process. We use cryptographic hashing which add salt, the random data to the input even more different and unique. 

generate_password_hash(password, rounds=None)
    This helper function wraps the eponymous method of :class:`Bcrypt`. It
    is intended to be used as a helper function at the expense of the
    configuration variable provided when passing back the app object. In other
    words this shortcut does not make use of the app object at all.

    To this this function, simple import it from the module and use it in a
    similar fashion as the method would be used. Here is a quick example::

        from flask.ext.bcrypt import generate_password_hash
        pw_hash = generate_password_hash('hunter2', 10)

    :param password: The password to be hashed.
    :param rounds: The optional number of rounds

>>> from flask.ext.bcrypt import generate_password_hash
__main__:1: ExtDeprecationWarning: Importing flask.ext.bcrypt is deprecated, use flask_bcrypt instead.
>>> from flask.bcrypt import generate_password_hash
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'flask.bcrypt'
>>> from flask_bcrypt import generate_password_hash
>>> help(generate_password_hash)

>>> generate_password_hash('secret')
b'$2b$12$QXeCz8VO58FW0nf/uyRbEuPOG/8JO/0FwAs/0UbRvcj0cwd8sTIai

- use from flask_bcrypt import generate_password_hash 
- hashed one starts with $2b$ or $2a$ 
- $12$ stands for the round of hashing
- the others are hash

>>> generate_password_hash('secret', 15)
b'$2b$15$1dxPwf3ywkPJde4EAn2LjOmDAMoEn3agrAoMP9IyUjudW.wIRdMBW'

** you can increase the round if the fail times of specific id is high. **
- it takes longer and trickier

>>> hashed_pw = generate_password_hash('secret')
>>> hashed_pw
b'$2b$12$8RVntUMKkRgOrs9K5TLh6./DGyhs2BdinJ8aD7EcRsvrDGC0erc/W'
>>> hashed_pw == generate_password_hash('secret')
False

- it puts salt in the hash process so it shows 'False'

>>> from flask_bcrypt import check_password_hash
>>> check_password_hash(hashed_pw, 'secret')
True

in /models.py

** classmethod describes a method that belongs to a class that can create the class it belongs to. **
- if we don't have classmethod we have to create a user instance to call create user which will create a user instance.



## Lecture : Before and After Requests
- set database for flask app
- handle connecting to or disconnecting from
- ** G object

we specified debug, port, host in app.run, I think it is a little bit cleaner to do these as variables. And then we chage them in one spot. Or if you start deploying to things like HEroku and you're gonna use the env variables. Now set them up here(upside of the app.py).

- g is global object, it gets passed around all the time inside Falsks, so we can use this to like, set up things that we want to have available everywhere.

in /app.py

<pre>
  1
  2 from flask import Flask, g
  3
  4 import models
  5
  6 DEBUG = True
  7 PORT = 8000
  8 HOST = '0.0.0.0'
  9
 10 app = Flask(__name__)
 11
 12
 13
 14 @app.before_request
 15 def before_request():
 16     """Connect to the database before each request."""
 17     g.db = models.DATABASE
 18     g.db.connect()
 19
 20
 21 @app.after_request
 22 def after_request(response):
 23     """Close the database connection after each request."""
 24     g.db.close()
 25     return response
 26
 27
 28
 29 if __name__ == '__main__':
 30
 31     app.run(debug=DEBUG, host=HOST, port=PORT)
 
 </pre>

## Lecture : Login Manager
- flask-login needs to be able to find teh current user or any user we're looking for. 
- to do this, we need to create a function that will return an instance of our user model. 

## Lecture : Flask-WTF Forms
- In Flask and in Django, people often get the wrong idea about forms.They hear the word form, and immediately think aboout HTML forms. They think that forms are all about display. And really that makes forms very limited. Forms are about validation. Making sure that your data matches a certain pattern. The de facto form lib for Flask is Flask-WTF
- pip install flask-wtf
- this package also provides us with CSRF(cross-site request forgery, protection)
- CSRF includes a custom one time code with each submission. And if the form doesn't have that code or doesn't have the right one, the request is ignored.

in /forms.py
<pre>
  2 from flask_wtf import Form
  3 from wtfomrs import StringField, PasswordField
  4 from wtforms.validators import (DataRequired, Regexp, ValidationError, Email
  5                                Length, EqualTo)
  6
  7 from models import User
  8
  9
 10 def name_exists(form, field):
 11     if User.select().where(User.username == field.data).exists():
 12         raise ValidationError("User with that name already exists.")
 13
 14
 15 def name_exists(form, field):
 16     if User.select().where(User.email == field.data).exists():
 17         raise ValidationError("User with that email already exists.")
 18
 19
 20 class RegisterForm(Form):
 21     username = StringField(
 22         'Username',
 23         validators=[
 24             DataRequired(),
 25             Regexp(
 26                 r'^{a-zA-Z0-9_]+$',
 27                 message=("Username should be one word, letters, numbers, "
 28                          "and underscores only.")
 29             ),
 30             name_exists
 31         ]) # validators are requirements, should be data specific format and no overlap
 32     email = StringField(
 33         'Email',
 34         validators=[
 35             DataRequired(),
 36             Email(),
 37             email_exists,
 38         ])
 39     password = PasswordField(
 40         'Password',
 41         validators=[
 42             DataRequired(),
 43             Length(min=2),
 44             EqualTo('password2', message='Passwords must match')
 45         ])
 46     password2 = PasswordField(
 47         'Confirm Password',
 48         validators=[DataRequired()]
 49         )
 
 </pre>

## Lecture : Registration View

- form.validate_on_submit() - When the form is submitted through POST, make sure the data is valid.
- Macro - A custom, executable bit of templating.
- form.hidden_tag() - Renders hidden fields inside of a hidden <div>.

In [1]:
class User():
    email = 'kadencho@gmail.com'

In [4]:
User.email.split('@')[0][0] + '*'*(len(User.email.split('@')[0])-1) + '@' \
+ User.email.split('@')[1]

'k*******@gmail.com'

In [5]:
len(User.email)

18

In [6]:
len('k*******@gmail.com')

18

In [7]:
User.email.split('@')[0][0] + '*'*(len(User.email.split('@')[0])-1) + '@' \
      + User.email.split('@')[1]

'k*******@gmail.com'

## Lecture : Login View

## Lecture : Logout View

login_user or logout_user are creating sessions on the user's browser. And they're giving them a cookie. And the cookie references their user account. And it says, this user's logger in. This isn't necessarily the absolute moset secure way of doing this. You can find other things on Flaks login.

- logout_user deletes that cookie, gets rid of that info. No longer know who they are.

## Lecture : Add some layout

## Lecture : Post Model
- like status update on tweets or facebook
- ForeignKeyField - A field that points to another database record.

## Lecture : Post Form and View

## Lecture : Stream View

http://peewee.readthedocs.io/en/latest/peewee/playhouse.html?highlight=pagination#PaginatedQuery